Google Colab is cool

In [1]:
!nvidia-smi

Sat Apr  5 08:29:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Check current versions
!python -c "import numpy; print('Current numpy version:', numpy.__version__)"
!python -c "import torch; print('Current torch version:', torch.__version__)"

# Install desired versions with CUDA 12.4
!pip install numpy==1.26.4
!pip install torch==2.3.0+cu124 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.3.0+cu124.html
!pip install torcheeg
!pip install pytorch-lightning

# This will show the versions in new Python processes
!python -c "import numpy; print('Installed numpy version:', numpy.__version__)"
!python -c "import torch; print('Installed torch version:', torch.__version__)"

print("\n*** IMPORTANT: Please restart the runtime now for changes to take effect in this notebook ***")

Current numpy version: 2.0.2
Current torch version: 2.6.0+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 77.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==2.3.0+cu124 (from versions: 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1.0+cu121.with.pypi

In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")

if torch.cuda.is_available():
    accelerator = "gpu"
    pin_memory = True
    print("GPU is available. Training will use GPU acceleration.")
else:
    accelerator = "cpu"
    pin_memory = False
    print("WARNING: GPU not available, using CPU instead")

PyTorch version: 2.6.0+cu124
CUDA version: 12.4
GPU is available. Training will use GPU acceleration.


In [2]:
from torcheeg.datasets import DREAMERDataset
from torcheeg import transforms
from torcheeg.datasets.constants import DREAMER_CHANNEL_LOCATION_DICT
from torcheeg.trainers import ClassifierTrainer

from torch.utils.data import DataLoader, random_split
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

import os

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
experiment_name = input("name your experiment > ")  # like ccnn_01, sst_01

name your experiment > sst_01


In [5]:
IO_PATH = f'/content/drive/MyDrive/datasets/DREAMER/io/{experiment_name}/' # not-existant or populated - ! NOT EMPTY !
MODEL_SAVE_PATH = f'/content/drive/MyDrive/datasets/DREAMER/model/{experiment_name}/' # has to exist
MAT_FILE_PATH = '/content/drive/MyDrive/datasets/DREAMER/DREAMER.mat' # has to exist

if not os.path.exists(MODEL_SAVE_PATH):
    print(f"model folder not existing, creating at {MODEL_SAVE_PATH}")
    os.mkdir(MODEL_SAVE_PATH)
else:
    print(f"ATTENTION: model folder already exists, only proceed if you are ready to overwrite")

if not os.path.exists(MAT_FILE_PATH):
    raise Exception("make sure MAT file exists")


SEED:int = 42
BATCH_SIZE:int = 64
N_WORKERS:int = 2

ATTENTION: model folder already exists, only proceed if you are ready to overwrite


#loading dataset and initializing model

In [6]:
# https://zenodo.org/records/546113
# Katsigiannis, S., & Ramzan, N. (2017). DREAMER: A Database for Emotion Recognition through EEG and ECG Signals from Wireless Low-cost Off-the-Shelf Devices [Data set]. Zenodo. https://doi.org/10.1109/JBHI.2017.2688239
# we predict arousal (intensity of emotion) and valence (negativity of emotion) on scale of 1-5, negative and aroused means fear (or masochism (me fr))

# if this is throwing an error about corrupted, make sure the folder is
# populated by the right stuff (from running this thing the last time)
# or does NOT exist

# will populate io_path - will only have to run once

# --- labeling
# valence should be smaller than 3 (negative emotions), arousal should be higher than 3
# here this will return an in [0, 1, 2, 3] for each possible combination of the two binaries
# for us it is fear, when [0, 1] (so 1)

label_transform = transforms.Compose([
        transforms.Select(['valence', 'arousal']),
        transforms.Binary(3),
        transforms.BinariesToCategory()
])


# --- CCNN (55 min)
# dataset = DREAMERDataset(
#     io_path=IO_PATH,
#     mat_path=MAT_FILE_PATH,

#     offline_transform=transforms.Compose([
#         transforms.BandDifferentialEntropy(apply_to_baseline=True),
#         transforms.ToGrid(DREAMER_CHANNEL_LOCATION_DICT, apply_to_baseline=True)
#     ]),

#     online_transform=transforms.Compose([
#         transforms.BaselineRemoval(),
#         transforms.ToTensor()
#     ]),

#     label_transform=label_transform,

#     num_worker=N_WORKERS
# )


# --- SST-EmotionNet
dataset = DREAMERDataset(
    io_path=IO_PATH,
    mat_path=MAT_FILE_PATH,

    offline_transform=transforms.Compose([
        transforms.BaselineRemoval(),
        transforms.Concatenate([
            transforms.Compose([
                transforms.BandDifferentialEntropy(sampling_rate=128),
                transforms.MeanStdNormalize()
            ]),
            transforms.Compose([
                transforms.Downsample(num_points=32),
                transforms.MinMaxNormalize()
            ])
        ]),
        transforms.ToInterpolatedGrid(DREAMER_CHANNEL_LOCATION_DICT)
    ]),

    online_transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((16, 16))
    ]),

    label_transform=label_transform,

    num_worker=N_WORKERS
)

[2025-04-05 09:02:29] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from /content/drive/MyDrive/datasets/DREAMER/io/sst_01/.
INFO:torcheeg:🔍 | Detected cached processing results, reading cache from /content/drive/MyDrive/datasets/DREAMER/io/sst_01/.


In [7]:
# validation is used during training for evaluating each batch
# test is saved to after (even if we don't train on val anyway, mechanisms like picking the best may still have 'val bias')
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(SEED)
)

print(f"Dataset split: {train_size} training, {val_size} validation, {test_size} test samples")



# when training a GPU, we put the memory on a special 'pinned' region for faster transfer between CPU and GPU
# we shuffle the data in training to avoid it learning some patterns there (across batches) - no need on eval, since metrics will be the same
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=N_WORKERS,
    pin_memory=pin_memory
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=N_WORKERS,
    pin_memory=pin_memory
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=N_WORKERS,
    pin_memory=pin_memory
)

# the grid:
# - channels mapped to a grid (not every one has an electrode)
# - each frequency band has its own grid, similar to how we have multiple values for RGB
# - determined by the DICT we enter into transform (though we might resize it)

for batch in train_loader:
    print(f"Input shape: {batch[0].shape}")
    break

# CNN: ([64, 4, 9, 9]) - ([batch_size, n_bands, grid_dim, grid_dim])
# SST ([64, 36, 16, 16])

Dataset split: 60020 training, 12861 validation, 12863 test samples
Input shape: torch.Size([64, 36, 16, 16])


In [8]:
# --- CCNN (45min)
# from torcheeg.models import CCNN
# model = CCNN(
#     num_classes=4,  # see dataset init
#     in_channels=4,  # 4 frequency bands
#     grid_size=(9, 9)
# )

# trainer = ClassifierTrainer(
#     model=model,
#     num_classes=4,
#     lr=1e-4,
#     weight_decay=1e-4,
#     accelerator=accelerator,
#     metrics=["accuracy", "precision", "recall", "f1score"]
# )


# --- SST (47min, stopped on epoch 35)
from torcheeg.models import SSTEmotionNet
model = SSTEmotionNet(
    temporal_in_channels=32, # 32 "frames" stacked
    spectral_in_channels=4, # 4 bands
    grid_size=(16, 16), # min size for some reason
    num_classes=4
)

trainer = ClassifierTrainer(
    model=model,
    num_classes=4,
    lr=5e-5,
    weight_decay=1e-4,
    accelerator=accelerator,
    metrics=["accuracy", "precision", "recall", "f1score"]
)


# actual training (don't run if you just want to transform existing)

In [ ]:
callbacks = [
    # saves the best model (for recalling after plateu)
    ModelCheckpoint(
        dirpath=MODEL_SAVE_PATH,
        filename='best-model-{epoch:02d}-{val_accuracy:.4f}',
        monitor='val_accuracy',
        mode='max',
        save_top_k=1
    ),

    ModelCheckpoint(
        dirpath=MODEL_SAVE_PATH,
        filename='last-model',
        save_last=True
    ),

    # detect overfitting and end if val_accuracy stops improving for 10 epochs
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        mode='max',
        verbose=True
    )
]

In [ ]:
print("\nStarting model training...")
trainer.fit(
    train_loader,
    val_loader,
    max_epochs=100,
    default_root_dir=MODEL_SAVE_PATH,
    callbacks=callbacks,
    enable_progress_bar=True,
    enable_model_summary=True
)

# ends once the trainer is done (so most likely through EarlyStop, otherwise max_epochs)


Starting model training...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/drive/MyDrive/datasets/DREAMER/model/sst_01 exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | SSTEmotionNet    | 525 K  | train
1 | ce_fn         | CrossEntropyLoss | 0      | train
2 | train_loss    | MeanMetric       | 0      | train
3 | val_loss      | MeanMetric       | 0      | train
4 | test_loss     | MeanMetric       | 0      | train
5 | train_metrics | MetricCollection | 0      | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[2025-04-02 07:46:27] INFO (torcheeg/MainThread) 
[Val] val_loss: 1.446 val_accuracy: 0.047 val_precision: 0.012 val_recall: 0.250 val_f1score: 0.022 
INFO:torcheeg:
[Val] val_loss: 1.446 val_accuracy: 0.047 val_precision: 0.012 val_recall: 0.250 val_f1score: 0.022 


Training: |          | 0/? [00:00<?, ?it/s]

...

[2025-04-02 08:31:16] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.909 val_accuracy: 0.735 val_precision: 0.703 val_recall: 0.692 val_f1score: 0.696 
INFO:torcheeg:
[Val] val_loss: 0.909 val_accuracy: 0.735 val_precision: 0.703 val_recall: 0.692 val_f1score: 0.696 
[2025-04-02 08:31:16] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.179 train_accuracy: 0.934 train_precision: 0.926 train_recall: 0.920 train_f1score: 0.923 
INFO:torcheeg:
[Train] train_loss: 0.179 train_accuracy: 0.934 train_precision: 0.926 train_recall: 0.920 train_f1score: 0.923 


Validation: |          | 0/? [00:00<?, ?it/s]

[2025-04-02 08:32:32] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.945 val_accuracy: 0.733 val_precision: 0.699 val_recall: 0.701 val_f1score: 0.698 
INFO:torcheeg:
[Val] val_loss: 0.945 val_accuracy: 0.733 val_precision: 0.699 val_recall: 0.701 val_f1score: 0.698 
[2025-04-02 08:32:33] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.173 train_accuracy: 0.936 train_precision: 0.929 train_recall: 0.923 train_f1score: 0.926 
INFO:torcheeg:
[Train] train_loss: 0.173 train_accuracy: 0.936 train_precision: 0.929 train_recall: 0.923 train_f1score: 0.926 


Validation: |          | 0/? [00:00<?, ?it/s]

[2025-04-02 08:33:50] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.943 val_accuracy: 0.748 val_precision: 0.740 val_recall: 0.679 val_f1score: 0.704 
INFO:torcheeg:
[Val] val_loss: 0.943 val_accuracy: 0.748 val_precision: 0.740 val_recall: 0.679 val_f1score: 0.704 
[2025-04-02 08:33:50] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.167 train_accuracy: 0.937 train_precision: 0.930 train_recall: 0.925 train_f1score: 0.928 
INFO:torcheeg:
[Train] train_loss: 0.167 train_accuracy: 0.937 train_precision: 0.930 train_recall: 0.925 train_f1score: 0.928 
INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric val_accuracy did not improve in the last 10 records. Best score: 0.748. Signaling Trainer to stop.


In [ ]:
print("\nEvaluating model on test set...")
test_results = trainer.test(test_loader)[0]
print(test_results)
print(f"Test accuracy: {test_results['test_accuracy']:.4f}")

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.



Evaluating model on test set...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

[2025-04-02 08:34:21] INFO (torcheeg/MainThread) 
[Test] test_loss: 0.930 test_accuracy: 0.753 test_precision: 0.737 test_recall: 0.679 test_f1score: 0.703 
INFO:torcheeg:
[Test] test_loss: 0.930 test_accuracy: 0.753 test_precision: 0.737 test_recall: 0.679 test_f1score: 0.703 


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7530125379562378     │
│       test_f1score        │    0.7026134133338928     │
│         test_loss         │    0.9296864867210388     │
│      test_precision       │    0.7367448806762695     │
│        test_recall        │    0.6794671416282654     │
└───────────────────────────┴───────────────────────────┘

{'test_loss': 0.9296864867210388, 'test_accuracy': 0.7530125379562378, 'test_precision': 0.7367448806762695, 'test_recall': 0.6794671416282654, 'test_f1score': 0.7026134133338928}
Test accuracy: 0.7530


#reloading model and exporting to onnx

In [17]:
!pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.9 MB/s eta 0:00:00


In [16]:
import os

for fname in os.listdir(MODEL_SAVE_PATH):
    if fname.startswith("best-model") and fname.endswith(".ckpt"):
        break


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


# continuing code will fail if for some reason there is no file called best-model-...ckpt
# (is automatically created due to our callbacks in training)
ckpt_path = MODEL_SAVE_PATH + fname

print(f"Loading {ckpt_path}")
checkpoint = torch.load(
    ckpt_path,
    map_location=device
)


if 'state_dict' in checkpoint:
    state_dict = {k.replace('model.', ''): v for k, v in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict)
else:
    model.load_state_dict(checkpoint)

Loading /content/drive/MyDrive/datasets/DREAMER/model/sst_01/best-model-epoch=25-val_accuracy=0.7485.ckpt


In [18]:
for batch in train_loader:
    dummy_input = batch[0][0:2] # take small batch of two (so the dynamic batching will work)
    print(dummy_input.shape) # torch.Size([2, 36, 16, 16])
    break



onnx_path = MODEL_SAVE_PATH + "inference.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=13,
    do_constant_folding=True,
    input_names=["eeg_input"],
    output_names=["class_probabilities"],
    dynamic_axes={
        "eeg_input": {0: "batch_size"},
        "class_probabilities": {0: "batch_size"}
    }
)

torch.Size([2, 36, 16, 16])


In [19]:
#
# loading model and doing inference to check if this works
#

import onnxruntime as ort
import numpy as np


session_options = ort.SessionOptions()
session_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] # fallbacks to CPU if CUDA isn't available


session = ort.InferenceSession(
    onnx_path,
    sess_options=session_options,
    providers=providers
)


input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


# numpy is more efficient that tensors, and in inference we don't need AutoGrad features
test_batch = next(iter(test_loader))
test_data, test_labels = test_batch[0], test_batch[1]
test_data_np = test_data.numpy()


start_time = torch.cuda.Event(enable_timing=True)
end_time = torch.cuda.Event(enable_timing=True)


onnx_outputs = session.run([output_name], {input_name: test_data_np})
probabilities = onnx_outputs[0]
predicted_classes = np.argmax(probabilities, axis=1)
print(predicted_classes)

/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


[3 0 3 3 1 2 1 3 3 3 3 1 2 1 2 2 2 3 1 1 1 3 3 0 1 3 1 1 3 2 1 3 3 3 3 3 3
 3 3 3 1 3 3 1 1 0 1 2 3 1 1 3 3 3 2 1 3 1 1 3 3 0 3 2]
